In [336]:
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from glob import glob

print(sys.executable)


/Users/aktan/opt/anaconda3/envs/env-analythics/bin/python


In [337]:
data_path = os.path.join(os.getcwd(), "data")
apple_data_path = os.path.join(data_path, "apple_health_data_from_csv")

### Read the file content 

In [338]:
apple_health_raw = list()
apple_health_workouts_raw = list()
avoid_those_types = {"SixMinuteWalkTestDistance", "MindfulSession"}
avoid_those_cols = {"HKExternalUUID", 
                    "HKMetadataKeyAppleDeviceCalibrated", "HKMetadataKeyHeartRateMotionContext",
                    "HKMetadataKeyDevicePlacementSide", "HKMetadataKeySyncIdentifier",
                    "HKMetadataKeySyncVersion", "HKSexualActivityProtectionUsed", 
                    "HKTimeZone"}

for f_name in os.listdir(apple_data_path):
    record_data = pd.read_csv(os.path.join(apple_data_path, f_name), skiprows=1, low_memory=False)
    if "Quantity" in f_name: 
        type_type = "continious"
        type_name = f_name.split("_")[0].replace("HKQuantityTypeIdentifier", "")
    if "Category" in f_name: 
        type_type = "categorical"
        type_name = f_name.split("_")[0].replace("HKCategoryTypeIdentifier", "")
    if "WorkoutActivity" in f_name:
        type_type = "workout"
        type_name = f_name.split("_")[0].replace("HKWorkoutActivityType", "")
    if type_name in avoid_those_types:
        continue
    record_data["type_type"] = type_type
    record_data["type"] = type_name
    wanted_cols = list(set(record_data.columns.values).difference(avoid_those_cols))
    if type_type == "workout":
        apple_health_workouts_raw.append(record_data[wanted_cols])
    else:
        apple_health_raw.append(record_data[wanted_cols])

apple_health_df = pd.concat(apple_health_raw)
apple_health_workouts_df = pd.concat(apple_health_workouts_raw)

In [339]:
apple_health_df.shape

(1092228, 18)

In [340]:
apple_health_workouts_df.shape

(591, 23)

### Format df, make it beatuful

* Format device column

In [341]:
def get_device_info(device_info, key):
    if pd.isna(device_info):
        return_dict =  {"device_HKDevice":None, "device_name": None, 
                "device_manufacturer":None, "device_model": None, "device_hardware":None, "device_software":None}
    else:
        device_info = device_info.replace("<<", "").replace(">", "")
        fields = list(filter(lambda x: len(x) == 2, [x.strip().split(":") for x in device_info.split(",")]))
        return_dict =  {"device_"+x:y for x, y in fields}
    return return_dict.get(key, -1)

# Format device info in apple_health_df 
apple_health_df["device_name"] = apple_health_df["device"].apply(lambda x: get_device_info(x, "device_name"))
apple_health_df["device_software"] = apple_health_df["device"].apply(lambda x: get_device_info(x, "device_software"))

apple_health_workouts_df["device_name"] = apple_health_workouts_df["device"].apply(lambda x: get_device_info(x, "device_name"))
apple_health_workouts_df["device_software"] = apple_health_workouts_df["device"].apply(lambda x: get_device_info(x, "device_software"))


In [342]:
apple_health_df["device_name"].unique()

array([None, 'iPhone', 'Apple Watch', 'EarPods', 'AirPods'], dtype=object)

In [343]:
apple_health_df["device_name"] = apple_health_df["device_name"].apply(lambda x: "Apple Watch" if pd.isna(x) else x)

In [344]:
apple_health_workouts_df["device_name"].unique()

array([None, 'Apple Watch'], dtype=object)

In [345]:
apple_health_workouts_df["device_name"] =  apple_health_workouts_df["device_name"].apply(lambda x: "Apple Watch" if pd.isna(x) else x)

* Filter out na valued rows

In [346]:
# Format health_df 
before_measurements = apple_health_df["type"].unique()
apple_health_df = apple_health_df.dropna(subset=["value"])
after_measurements = apple_health_df["type"].unique()
print("This measures will be removed, after removing na valued")
print(set(before_measurements).difference(set(after_measurements)))

This measures will be removed, after removing na valued
set()


In [347]:
print("{0} of rows left removing null values".format(len(apple_health_df)))

1092228 of rows left removing null values


* Taking a look at data types and sizes so we set a threshold # of records threshold.

In [348]:
apple_health_df.groupby(["type"])["value"].count().sort_values(ascending=False)

type
ActiveEnergyBurned                390353
BasalEnergyBurned                 196237
HeartRate                         154937
DistanceWalkingRunning            129296
StepCount                          95550
AppleExerciseTime                  18123
AppleStandTime                     16030
EnvironmentalAudioExposure         11446
RespiratoryRate                     9556
FlightsClimbed                      8849
BodyMass                            8324
BodyMassIndex                       8320
WalkingSpeed                        6206
WalkingStepLength                   6204
AppleStandHour                      6084
WalkingDoubleSupportPercentage      5424
OxygenSaturation                    3836
HeadphoneAudioExposure              3769
SleepAnalysis                       3488
HeartRateVariabilitySDNN            2439
WalkingAsymmetryPercentage          2388
StairDescentSpeed                   1796
StairAscentSpeed                    1259
HandwashingEvent                     965
RestingHear

In [349]:
data_catalogue = apple_health_df.groupby(["type"])["value"].count().sort_values(ascending=False).reset_index()
columns = data_catalogue[data_catalogue["value"] >= 35]["type"].values.tolist()
columns.remove("MenstrualFlow")
columns.remove("HandwashingEvent")

In [351]:
apple_health_df = apple_health_df[apple_health_df["type"].isin(columns)].copy()

* Dealing with the time values 

In [352]:
def get_time_difference(time_1, time_2, in_terms_of="h"):
    """
    :param time_1: str in such a format 2021-10-25 10:21:39 +0300
    :param time_2: str in such a format 2021-10-25 10:21:39 +0300
    :param in_terms_of: "h" for hours "d" for days
    """
    dt_time_1 = datetime.strptime(time_1, "%Y-%m-%d %H:%M:%S %z")
    dt_time_2 = datetime.strptime(time_2, "%Y-%m-%d %H:%M:%S %z")
    difference = (dt_time_1-dt_time_2)
    
    if in_terms_of == "h":
        return int(np.floor(difference.seconds/60/60))
    elif in_terms_of == "d":
        return int(np.floor(difference.seconds/60/60/24))

def get_date(time_1, return_type="str"):
    """
    :param time_1: str in such a format 2021-10-25 10:21:39 +0300
    :param return_type: str or datetime
    """
    dt_time_1 = datetime.strptime(time_1, "%Y-%m-%d %H:%M:%S %z").date()
    if return_type == "str":
        return dt_time_1.strftime("%Y-%m-%d")
    else:
        dt_time_1

In [353]:
apple_health_df["start_date"] = apple_health_df["startDate"].apply(lambda x: get_date(x))
apple_health_df["end_date"] = apple_health_df["endDate"].apply(lambda x: get_date(x))

apple_health_workouts_df["start_date"] = apple_health_workouts_df["startDate"].apply(lambda x: get_date(x))
apple_health_workouts_df["end_date"] = apple_health_workouts_df["endDate"].apply(lambda x: get_date(x))

### Drop Unnecesary columns 

* First take a look at them 

In [354]:
for col in apple_health_df.columns:
    print(col, "na_percent: ", apple_health_df[col].isna().sum() / len(apple_health_df) * 100)

value na_percent:  0.0
startDate na_percent:  0.0
HKWasUserEntered na_percent:  99.99954178465713
productType na_percent:  0.46270585324279
device na_percent:  4.168384974202477
endDate na_percent:  0.0
type_type na_percent:  0.0
type na_percent:  0.0
sourceName na_percent:  0.0
sourceVersion na_percent:  0.46701307746588583
unit na_percent:  0.8772074524143367
HKAlgorithmVersion na_percent:  99.7764825557419
HKMenstrualCycleStart na_percent:  100.0
flags na_percent:  100.0
HKMetadataKeyBarometricPressure na_percent:  99.64845718894051
streaks_date na_percent:  99.98423739220485
streaks_entry_id na_percent:  99.98423739220485
HKVO2MaxTestType na_percent:  99.98845297335936
device_name na_percent:  0.0
device_software na_percent:  4.168384974202477
start_date na_percent:  0.0
end_date na_percent:  0.0


In [355]:
apple_health_df = apple_health_df.drop(["startDate", "endDate", "device"], axis=1).copy()

In [356]:
for col in apple_health_workouts_df.columns:
    print(col, "na_percent: ", apple_health_workouts_df[col].isna().sum() / len(apple_health_workouts_df) * 100)

totalFlightsClimbed na_percent:  100.0
duration na_percent:  0.0
startDate na_percent:  0.0
activityType na_percent:  0.0
productType na_percent:  0.0
durationUnit na_percent:  0.0
device na_percent:  8.460236886632826
endDate na_percent:  0.0
totalDistance na_percent:  47.71573604060914
totalEnergyBurned na_percent:  2.3688663282571913
type_type na_percent:  0.0
type na_percent:  0.0
totalSwimmingStrokeCount na_percent:  100.0
sourceName na_percent:  0.0
sourceVersion na_percent:  0.0
HKAverageMETs na_percent:  8.629441624365482
HKWeatherTemperature na_percent:  21.99661590524535
HKElevationAscended na_percent:  71.74280879864637
HKIndoorWorkout na_percent:  7.2758037225042305
HKWeatherHumidity na_percent:  21.99661590524535
HKGroupFitness na_percent:  97.96954314720813
HKCoachedWorkout na_percent:  97.96954314720813
HKWorkoutBrandName na_percent:  97.96954314720813
device_name na_percent:  0.0
device_software na_percent:  8.460236886632826
start_date na_percent:  0.0
end_date na_perc

In [357]:
apple_health_workouts_df = apple_health_workouts_df.drop(
    ['startDate', 'endDate', "totalFlightsClimbed", "totalSwimmingStrokeCount", "device"], axis=1).copy()

### Make measure value columns float 

* In apple_health_df

In [358]:
apple_health_df["value"] = apple_health_df.apply(lambda x: float(x["value"]) if x["type_type"] == "continious" else x["value"],
                                                 axis=1)

* In workouts df

In [359]:
measure_values = ["duration", "totalDistance", "totalEnergyBurned", "HKElevationAscended"]
measure_vs_unit = {"totalEnergyBurned": "kcal", "totalDistance": "m", "HKElevationAscended": "cm"}

In [360]:
apple_health_workouts_df["totalEnergyBurned"] = apple_health_workouts_df["totalEnergyBurned"].apply(
    lambda x: float(str(x).replace(measure_vs_unit.get("totalEnergyBurned"), "")))

In [361]:
apple_health_workouts_df["totalDistance"] = apple_health_workouts_df["totalDistance"].apply(
    lambda x: float(str(x).replace(measure_vs_unit.get("totalDistance"), "")))

In [362]:
apple_health_workouts_df["HKElevationAscended"] = apple_health_workouts_df["HKElevationAscended"].apply(
    lambda x: float(str(x).replace(measure_vs_unit.get("HKElevationAscended"), "")))

In [363]:
apple_health_workouts_df["duration"] = apple_health_workouts_df["duration"].apply(
    lambda x: float(x))

In [364]:
apple_health_workouts_df.shape

(591, 22)

### Group by 

In [365]:
apple_health_df_agg_cont = apple_health_df[apple_health_df["type_type"] == "continious"].groupby(
    ["end_date", "type", "type_type", "device_name"]).agg({"value": "sum"}).reset_index()
print(len(apple_health_df_agg_cont))

14979


In [366]:
apple_health_df_agg_cat= apple_health_df[apple_health_df["type_type"] == "categorical"].groupby(
    ["end_date", "type", "type_type", "device_name"]).agg({"value": list}).reset_index()
print(len(apple_health_df_agg_cat))


799


* Format categorical values 

In [372]:
def get_freq(a_series):
    
    instance, freq = np.unique(a_series["value"], return_counts=True)
    if a_series["type"] == "SleepAnalysis":
        return dict(zip(instance, freq)).get("inBed", 0)
    elif a_series["type"] == "AppleStandHour":
        return dict(zip(instance, freq)).get("stood", 0)
        

apple_health_df_agg_cat["value"] = apple_health_df_agg_cat.apply(lambda x: get_freq(x), axis=1)

In [373]:
apple_health_df_agg_cat.head()

,end_date,type,type_type,device_name,value
0,2016-11-07,SleepAnalysis,categorical,iPhone,1
1,2016-11-08,SleepAnalysis,categorical,iPhone,2
2,2016-11-09,SleepAnalysis,categorical,iPhone,7
3,2016-11-10,SleepAnalysis,categorical,iPhone,6
4,2016-11-11,SleepAnalysis,categorical,iPhone,2


### Concat dfs 

In [374]:
apple_health_final_df = pd.concat([apple_health_df_agg_cont, apple_health_df_agg_cat]).reset_index()
apple_health_final_df.shape

(15778, 6)

### Order by date 

In [375]:
apple_health_final_df = apple_health_final_df.sort_values(by='end_date')

In [377]:
apple_health_workouts_df = apple_health_workouts_df.sort_values(by='end_date')

### Export Raw

In [378]:
apple_health_final_df.to_csv("apple_health_data_ece.csv")
apple_health_workouts_df.to_csv("apple_excersize_data_ece.csv")

### Further format health data 

In [379]:
apple_health_final_df.sample(10)

,index,end_date,type,type_type,device_name,value
13349,13349,2022-07-26,FlightsClimbed,continious,iPhone,15.0
10366,10366,2022-03-21,RespiratoryRate,continious,Apple Watch,704.5
1412,1412,2017-07-16,FlightsClimbed,continious,iPhone,19.0
14338,14338,2022-09-06,WalkingSpeed,continious,iPhone,29.706514
4595,4595,2020-08-04,StepCount,continious,iPhone,1546.0
11272,11272,2022-04-28,RestingHeartRate,continious,Apple Watch,53.0
12304,12304,2022-06-12,FlightsClimbed,continious,iPhone,19.0
12724,12724,2022-06-28,StairDescentSpeed,continious,Apple Watch,7.507551
1545,1545,2017-09-03,DistanceWalkingRunning,continious,iPhone,4.156421
11843,11843,2022-05-23,VO2Max,continious,Apple Watch,69.81


In [381]:
apple_health_workouts_df.sample(5)

,duration,activityType,productType,durationUnit,totalDistance,totalEnergyBurned,type_type,type,sourceName,sourceVersion,...,HKElevationAscended,HKIndoorWorkout,HKWeatherHumidity,HKGroupFitness,HKCoachedWorkout,HKWorkoutBrandName,device_name,device_software,start_date,end_date
105,2744.717232,FunctionalStrengthTraining,"Watch6,6",sec,NaN,131.7040,workout,FunctionalStrengthTraining,PINAR ECE ’s Apple Watch,8.4.2,...,NaN,0.0,NaN,NaN,NaN,NaN,Apple Watch,8.4.2,2022-04-09,2022-04-09
1,291.779880,FunctionalStrengthTraining,"Watch6,6",sec,NaN,27.1950,workout,FunctionalStrengthTraining,PINAR ECE ’s Apple Watch,8.0,...,NaN,0.0,6900 %,NaN,NaN,NaN,Apple Watch,8.0,2021-10-25,2021-10-25
26,1838.758451,JumpRope,"Watch6,6",sec,1700.980,196.6670,workout,JumpRope,PINAR ECE ’s Apple Watch,8.6,...,NaN,0.0,8800 %,NaN,NaN,NaN,Apple Watch,8.6,2022-06-16,2022-06-16
207,386.443490,Walking,"Watch6,6",sec,322.077,35.9677,workout,Walking,PINAR ECE ’s Apple Watch,8.6,...,2469.0,0.0,8000 %,NaN,NaN,NaN,Apple Watch,8.6,2022-06-24,2022-06-24
19,227.888627,JumpRope,"Watch6,6",sec,146.092,16.8240,workout,JumpRope,PINAR ECE ’s Apple Watch,8.6,...,NaN,0.0,7700 %,NaN,NaN,NaN,Apple Watch,8.6,2022-06-11,2022-06-11
